<a href="https://colab.research.google.com/github/steliosg23/PDS-A2/blob/main/Incidents%20Augmentation%20using%20techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Augmenting the Initial Imbalanced Train Set for Benchmark and Submission Models

In this notebook, **"Incidents Train Set Augmentation"**, I will address the issue of class imbalance in the initial training dataset for the **Food Hazard Detection Challenge**. The augmentation process aims to create a more balanced and diverse dataset to improve the model's performance in detecting food safety-related incidents accurately. However, for practical reasons, the size of the dataset will be approximately tripled. While this improves class representation, some imbalance will still persist to maintain a manageable dataset size and training efficiency. This augmented dataset will be utilized in two critical notebooks, as outlined below:

1. **Benchmark Models Finetuned PubMedBERT**:
   - The augmented dataset will serve as the foundation for training and evaluating benchmark models in the notebook titled **"AnB Benchmark PubMedBERT.ipynb"**. These models will establish baseline performance metrics for each classification task, helping us understand the impact of data augmentation on model learning and generalization. This evaluation will include hazard-category, product-category, hazard, and product classifications.

2. **Submission Model Finetuned PubMedBERT**:
   - The augmented dataset will also be used in the notebook titled **AnB Submission PubMedBERT.ipynb"** to train the final submission model. This model will be fine-tuned on the enhanced dataset to maximize its accuracy and effectiveness in making predictions for the competition. The submission model will represent the culmination of our efforts in preprocessing, augmentation, and fine-tuning to achieve optimal performance on the challenge leaderboard.

By approximately doubling the dataset size and improving the representation of minority classes, this notebook sets the stage for robust training. While some class imbalance will still remain, this approach balances the trade-off between practical dataset size and improved model performance, trying to operate a more effective training for both benchmark and submission models.


# Mount Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import Required Libraries
This cell imports the libraries necessary for the project:
- `pandas`: For data manipulation and analysis.
- `Counter` from `collections`: To count occurrences of hazard and product classes.
- `MarianMTModel` and `MarianTokenizer` from `transformers`: For translation models used in back-translation.
- `wordnet` from `nltk`: For obtaining synonyms of words.
- `torch`: For GPU support and managing computations.
- `random`: For random operations in augmentations.
- `nltk`: For natural language processing tasks.


In [ ]:
import pandas as pd
from transformers import pipeline
from torch.utils.data import DataLoader, Dataset
from nltk.corpus import wordnet
import torch
import nltk
from tqdm import tqdm

# Download NLTK WordNet
The WordNet lexical database is required to find synonyms during text augmentation. This step ensures it's downloaded and ready for use.


In [ ]:
nltk.download('wordnet')


True

# Check if GPU Support is Available
This step checks whether a GPU is available and sets the device accordingly. If a GPU is available, it will be used for computations to speed up model processing.


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Load the Dataset
The dataset is loaded from your Google Drive into a Pandas DataFrame for analysis. The file path is specified, and the dataset is assumed to be in CSV format.


In [ ]:
file_path = '/content/drive/MyDrive/Data/incidents_train.csv'
df = pd.read_csv(file_path)

# Set a target threshold for balancing
To ensure balanced datasets for classification tasks, a target threshold is defined. This threshold helps limit the number of examples per class for specific target columns to improve model performance and prevent class imbalance issues. The selected columns are:

- `hazard`
- `product`
- `hazard-category`
- `product-category`

The threshold value is set to `100`, meaning that the number of examples per class in these columns will be capped at this value during preprocessing.


In [ ]:
# Set a target threshold for balancing
TARGET_THRESHOLD = 30
TARGET_COLUMNS = ['hazard', 'product']



# Translation Pipeline and Dataset Class
This section includes the implementation of a translation pipeline for English-French and French-English translations, utilizing the `Helsinki-NLP/opus-mt` models. The pipeline dynamically selects GPU or CPU based on availability to optimize performance.

Additionally, a custom `Dataset` class is defined for batch processing of text data, which is essential for efficient translation and further processing of large datasets.

In [ ]:
# Load translation pipeline
def load_translation_pipeline(source_lang="en", target_lang="fr"):
    print(f"Loading translation pipeline for {source_lang} → {target_lang}...")
    pipeline_instance = pipeline(
        "translation",
        model=f"Helsinki-NLP/opus-mt-{source_lang}-{target_lang}",
        tokenizer=f"Helsinki-NLP/opus-mt-{source_lang}-{target_lang}",
        device=0 if torch.cuda.is_available() else -1,
    )
    print(f"Translation pipeline for {source_lang} → {target_lang} loaded successfully.")
    return pipeline_instance

translate_en_to_fr = load_translation_pipeline("en", "fr")
translate_fr_to_en = load_translation_pipeline("fr", "en")

# Dataset class for batch processing
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

Loading translation pipeline for en → fr...


Translation pipeline for en → fr loaded successfully.
Loading translation pipeline for fr → en...
Translation pipeline for fr → en loaded successfully.


# Back-Translation with Batch Processing
To augment datasets and enhance model robustness, a back-translation function is implemented. This process translates input texts to a target language (French) and back to the source language (English). The function supports batch processing using PyTorch's `DataLoader` for efficient handling of large datasets.

Key Features:
- Batch size and maximum sequence length can be configured.
- Long texts are truncated to the specified `max_length`.
- Translations are performed using the pre-loaded pipelines for English-French and French-English translations.

In [ ]:
# Back-translation with batch processing
def back_translate_batch(texts, batch_size=16, max_length=512):
    print(f"Starting back-translation for {len(texts)} texts...")
    dataset = TextDataset(texts)
    dataloader = DataLoader(dataset, batch_size=batch_size)

    back_translated_texts = []
    for batch in tqdm(dataloader, desc="Back-translation"):
        inputs = translate_en_to_fr.tokenizer(batch, return_tensors="pt", truncation=True, padding=True, max_length=max_length).to(device)
        with torch.no_grad():
            translated_tokens = translate_en_to_fr.model.generate(**inputs)
        fr_texts = translate_en_to_fr.tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

        back_inputs = translate_fr_to_en.tokenizer(fr_texts, return_tensors="pt", truncation=True, padding=True, max_length=max_length).to(device)
        with torch.no_grad():
            back_translated_tokens = translate_fr_to_en.model.generate(**back_inputs)
        en_texts = translate_fr_to_en.tokenizer.batch_decode(back_translated_tokens, skip_special_tokens=True)

        back_translated_texts.extend(en_texts)

    print("Back-translation completed.")
    return back_translated_texts


# Data Augmentation and Balancing for Target Classes

To handle class imbalances in the dataset, two key techniques are applied:

1. **Data Augmentation for Underrepresented Classes**  
   Underrepresented classes are augmented to reach a specified target count using back-translation. This ensures diversity in the generated data while maintaining the original context.

2. **Downsampling Overrepresented Classes**  
   Overrepresented classes are downsampled to 60% of the initial size. This reduces the dominance of larger classes, ensuring balanced datasets.

These processes are applied iteratively across all target columns to create a balanced dataset for classification tasks.


In [ ]:
# Augment underrepresented classes
def augment_class(rows, target_count, batch_size=16):
    augmented_rows = []
    current_count = len(rows)
    texts = rows['text'].tolist()
    print(f"Augmenting class with {current_count} rows to reach {target_count} rows...")

    while current_count < target_count:
        new_texts = back_translate_batch(texts, batch_size=batch_size)
        augmented_rows.extend([{**row, 'text': text} for row, text in zip(rows.to_dict('records'), new_texts)])
        current_count += len(new_texts)
        print(f"Augmented rows: {current_count}/{target_count}")

    print(f"Class augmentation completed. Total rows: {current_count}.")
    return augmented_rows

# Downsample overrepresented classes
def downsample_class(rows, target_count):
    target_count = int(target_count)  # Convert to integer
    print(f"Downsampling class from {len(rows)} rows to {target_count} rows...")
    downsampled = rows.sample(n=target_count, random_state=42)
    print("Downsampling completed.")
    return downsampled


# Balancing Classes for Target Columns

To address class imbalances for each target column, the following steps are performed:

1. **Identify Underrepresented and Overrepresented Classes**  
   Class distributions are analyzed to identify classes with counts below or above the specified `TARGET_THRESHOLD`.

2. **Augment Underrepresented Classes**  
   Rows from underrepresented classes are augmented using back-translation to reach the target threshold. This enriches the dataset with diverse examples for these classes.

3. **Downsample Overrepresented Classes**  
   Rows from overrepresented classes are downsampled to 60% to reduce their dominance and ensure class balance if the target threshold is met .

This process is applied iteratively for all specified target columns, resulting in a balanced dataset that aims to improve the model's ability to handle class imbalances effectively.


In [ ]:
!pip install papermill
!nohup papermill data_augmentation.ipynb output_augmented.ipynb &


nohup: appending output to 'nohup.out'


In [ ]:
balanced_data = []
print("Starting dataset balancing...")

for column in TARGET_COLUMNS:
    print(f"Processing column: {column}")
    counts = df[column].value_counts()
    underrepresented_classes = counts[counts < TARGET_THRESHOLD].index.tolist()
    overrepresented_classes = counts[counts > TARGET_THRESHOLD].index.tolist()

    # Process underrepresented classes
    for cls in tqdm(underrepresented_classes, desc=f"Augmenting classes in {column}"):
        class_rows = df[df[column] == cls]
        augmented_data = augment_class(class_rows, TARGET_THRESHOLD)
        balanced_data.extend(augmented_data)


Starting dataset balancing...
Processing column: hazard


Augmenting class with 29 rows to reach 30 rows...
Starting back-translation for 29 texts...


Back-translation completed.
Augmented rows: 58/30
Class augmentation completed. Total rows: 58.
Augmenting class with 29 rows to reach 30 rows...
Starting back-translation for 29 texts...


Back-translation completed.
Augmented rows: 58/30
Class augmentation completed. Total rows: 58.
Augmenting class with 28 rows to reach 30 rows...
Starting back-translation for 28 texts...


Back-translation completed.
Augmented rows: 56/30
Class augmentation completed. Total rows: 56.
Augmenting class with 28 rows to reach 30 rows...
Starting back-translation for 28 texts...


...

Back-translation completed.
Augmented rows: 18/30
Starting back-translation for 3 texts...


Back-translation completed.
Augmented rows: 21/30
Starting back-translation for 3 texts...


Back-translation completed.
Augmented rows: 24/30
Starting back-translation for 3 texts...


Back-translation completed.
Augmented rows: 27/30
Starting back-translation for 3 texts...


Back-translation completed.
Augmented rows: 30/30
Class augmentation completed. Total rows: 30.
Augmenting class with 3 rows to reach 30 rows...
Starting back-translation for 3 texts...


**Due to RAM limitations the whole process of back translation and augmentation  was applied only for the 100% of hazard classes that were underbalanced (below 30 rows for each class) and 34% of the product classes that were also unbalanced the same way as hazard classes. Respectively, the downsampling (to 60% on initial size) was also applied as it follows for all classes that had over 30 rows for each class .**

In [ ]:
    # Process overrepresented classes
    for cls in tqdm(overrepresented_classes, desc=f"Downsampling classes in {column}"):
        class_rows = df[df[column] == cls]
        downsampled_data = downsample_class(class_rows, int(len(class_rows) * 0.6))  # Ensure target count is an integer
        balanced_data.extend(downsampled_data.to_dict('records'))


Downsampling class from 185 rows to 111 rows...
Downsampling completed.
Downsampling class from 138 rows to 82 rows...
Downsampling completed.
Downsampling class from 93 rows to 55 rows...
Downsampling completed.
Downsampling class from 79 rows to 47 rows...
Downsampling completed.
Downsampling class from 78 rows to 46 rows...
Downsampling completed.
Downsampling class from 74 rows to 44 rows...
Downsampling completed.
Downsampling class from 55 rows to 33 rows...
Downsampling completed.
Downsampling class from 52 rows to 31 rows...
Downsampling completed.
Downsampling class from 50 rows to 30 rows...
Downsampling completed.
Downsampling class from 50 rows to 30 rows...
Downsampling completed.
Downsampling class from 49 rows to 29 rows...
Downsampling completed.
Downsampling class from 45 rows to 27 rows...
Downsampling completed.
Downsampling class from 43 rows to 25 rows...
Downsampling completed.
Downsampling class from 42 rows to 25 rows...
Downsampling completed.
Downsampling clas

In [ ]:
balanced_df = pd.DataFrame(balanced_data)

# Saving and Validating the Balanced Dataset

Once the data balancing process is complete, the final balanced dataset is saved and validated through the following steps:

1. **Save the Balanced DataFrame**  
   The processed and balanced dataset is saved to a specified location as a CSV file. This ensures the results are preserved and can be reused for further analysis or model training.

2. **Preview the Balanced DataFrame**  
   Displaying the first few rows of the balanced dataset provides a quick validation of the balancing process and the augmented content.

3. **Verify Class Distributions**  
   Final class counts for each target column are printed to confirm that all classes meet the specified target threshold, ensuring the dataset is balanced and ready for downstream tasks.


In [ ]:
# Save and verify the balanced dataset
output_file = '/content/drive/MyDrive/Data/augmented_incidents_train.csv'
print(f"Saving balanced dataset to {output_file}...")
balanced_df.to_csv(output_file, index=False)
print("Dataset saved successfully.")

print("Verifying final class counts:")
for column in TARGET_COLUMNS:
    print(f"Final Counts for {column}:\n", balanced_df[column].value_counts())

Saving balanced dataset to /content/drive/MyDrive/Data/augmented_incidents_train.csv...
Dataset saved successfully.
Verifying final class counts:
Final Counts for hazard:
 hazard
listeria monocytogenes                        1118
salmonella                                    1029
milk and products thereof                     1027
escherichia coli                               405
eggs and products thereof                      392
                                              ... 
dairy products                                  29
organochlorine                                  29
paralytic shellfish poisoning (psp) toxins      29
product category/characteristics                29
hazelnut                                        29
Name: count, Length: 128, dtype: int64
Final Counts for product:
 product
ice cream                    151
chicken based products       146
cakes                         78
ready to eat - cook meals     74
beer                          69
                     

In [ ]:
balanced_df


,Unnamed: 0,year,month,day,country,title,text,hazard-category,product-category,hazard,product
0,351,2008,11,7,au,Country Cuisine Pty Ltd—Malouf’s Spice Mezza,"PRA No. 2008/10424 Date Published Nov 7, 2008 ...",allergens,herbs and spices,nuts,spice mix
1,555,2011,7,11,au,Haigh's Manufacturing Pty Ltd—Haigh’s Aprichoc...,"PRA No. 2011/12730 Publication Date Jul 11, 20...",allergens,"cocoa and cocoa preparations, coffee and tea",nuts,chocolate
2,611,2012,2,21,au,Coles Supermarkets Limited—Coles Deli 200g Spi...,"PRA No. 2012/13032 Publication Date Feb 21, 20...",allergens,"soups, broths, sauces and condiments",nuts,sauce
3,752,2012,12,4,us,2012 - Price Chopper Supermarkets Recalls Cent...,"FOR IMMEDIATE RELEASE - October 21, 2012 - (Sc...",allergens,cereals and bakery products,nuts,cakes
4,1000,2014,4,10,au,Coles Easter Eggs,Coles Supermarkets Australia Pty Ltd recalled ...,allergens,"cocoa and cocoa preparations, coffee and tea",nuts,chocolate eggs
...,...,...,...,...,...,...,...,...,...,...,...
11834,1902,2016,9,29,au,Quality Bakers Australia Pty Limited — Various...,PRA No. 2016/15657 Date published 29 Sep 2016 ...,foreign bodies,cereals and bakery products,metal fragment,bread
11835,1846,2016,8,18,au,Gluten Free Bakehouse Pty Ltd — Various Zehnde...,PRA No. 2016/15603 Date published 18 Aug 2016 ...,allergens,cereals and bakery products,soybeans and products thereof,bread
11836,3206,2018,11,30,us,Tres Hermanos Bakery Issues Allergy Alert on U...,"Wyoming, MI - Tres Hermanos Bakery of Wyoming,...",allergens,cereals and bakery products,milk and products thereof,bread
11837,243,2005,9,27,au,Gold Coast Bakery Queensland Pty Ltd—Vogels—Fr...,PRA No. 2005/8073 Date published 27 Sep 2005 P...,foreign bodies,cereals and bakery products,plastic fragment,bread
